# Project Phase 1: Video Dialog 

## Imports

In [ ]:
import json
from pprint import pprint

#Open Search
from opensearchpy import OpenSearch
from opensearchpy import helpers

#Embeddings neighborhood
from transformers import AutoTokenizer, AutoModel
import torch
import torch.nn.functional as F
import pickle
import spacy

#Contextual embeddings and self-attention
import numpy as np
from sklearn.manifold import TSNE
from sklearn.decomposition import PCA
import transformers
from transformers import AutoConfig, AutoModelForSequenceClassification
from bertviz import model_view, head_view

# Get the interactive Tools for Matplotlib
#%matplotlib notebook
#%matplotlib inline
import matplotlib.pyplot as plt
plt.style.use('ggplot')
import seaborn as sns


## 2.2 Text-based Search

### Load the video captions

In [ ]:
def load_captions_data(file_path):
    with open(file_path, 'r') as f:
        data = json.load(f)
    
    processed = {}
    for video_id, captions in data.items():
        processed[video_id] = {
            "segments": captions['segments'] if 'segments' in captions else captions,
        }
    return processed

# Load the data
val_data1 = load_captions_data('captions/val_1.json')
val_data2 = load_captions_data('captions/val_2.json')

# Combine dictionaries (preserving video_id as keys)
all_captions_data = {**val_data1, **val_data2}

pprint(f"Number of captions: {len(all_captions_data)}")
pprint(f"Example Captions: {all_captions_data}")

### Load the videos

In [ ]:
with open('activity_net.v1-3.min.json', 'r') as json_data:
    data = json.load(json_data)

database = {}

for video_id in data['database']:
    database["v_" + video_id] = data['database'][video_id]

# Criar lista ordenada com todos os dados completos
sorted_database = sorted(
    database.items(),
    key=lambda x: len(x[1]['annotations']),
    reverse=True
)

# Top 10 videos 
top_videos = dict(sorted_database[:27])

pprint(top_videos)

In [ ]:
matching_ids = set(database.keys()) & set(all_captions_data.keys())
print(f"Número de IDs correspondentes: {len(matching_ids)}")
print(f"IDs no top_videos: {list(top_videos.keys())[:5]}...")
print(f"IDs em all_captions_data: {list(all_captions_data.keys())[:5]}...")

### Compute the final captions dataset

In [ ]:
final_dataset_captions = {}

for video_id in top_videos:
    try:
        if (all_captions_data[video_id] != None):
            #final_dataset_video[video_id] = top_videos[video_id]
            final_dataset_captions[video_id] = all_captions_data[video_id]
    except Exception as e:
        None

final_dataset_captions.pop("v_PJ72Yl0B1rY", None)

pprint(final_dataset_captions)
pprint(len(final_dataset_captions))

### OpenSearch connection settings

In [ ]:
host = 'localhost'
port = 9200

user = 'admin' # Add your user name here.
password = 'grupo09FTC!' # Add your user password here. For testing only. Don't store credentials in code. 
index_name = user

Test if OpenSearch is up and running

In [ ]:
# Create the client with SSL/TLS enabled, but hostname verification disabled.
client = OpenSearch(
    hosts=[{'host': host, 'port': port}],
    http_auth=(user, password),
    use_ssl=True,              
    verify_certs=False,        
    ssl_show_warn=False
)

if client.indices.exists(index_name):

    resp = client.indices.open(index = index_name)
    print(resp)

    print('\n----------------------------------------------------------------------------------- INDEX SETTINGS')
    settings = client.indices.get_settings(index = index_name)
    pprint(settings)

    print('\n----------------------------------------------------------------------------------- INDEX MAPPINGS')
    mappings = client.indices.get_mapping(index = index_name)
    pprint(mappings)

    print('\n----------------------------------------------------------------------------------- INDEX #DOCs')
    print(client.count(index = index_name))
else:
    print("Index does not exist.")

In [ ]:
client.indices.delete(index=index_name, ignore=[400, 404])

### Create the index mappings for video captions

In [ ]:
index_body = {
    "settings": {
        "index": {
            "number_of_replicas": 0,
            "number_of_shards": 4,
            "refresh_interval": "-1",
            "knn": "true"
        },
    },
    "mappings": {
        "dynamic": "strict",
        "properties": {
            #video_id
            "title": {
                "type": "keyword"
            },
            #sentences
            "description": {
                "type": "text"
            },
        }
    }
}

if client.indices.exists(index=index_name):
    print("Index already exists.")
else:        
    response = client.indices.create(index_name, body=index_body)
    print('\nCreating index...')
    print(response)

Index the dataset

In [ ]:
for video_id, data in final_dataset_captions.items():
    print(f"Title: {video_id}")
    print(f"Description: {data['segments']['sentences']}")

for video_id, data in final_dataset_captions.items():
    filtered_caption = {
        "title": video_id,
        "description": data['segments']['sentences']
    }
    
    resp = client.index(index=index_name, id=video_id, body=filtered_caption)
    print(resp['result'])

### Search Functionality

Text-based Search 

In [ ]:
client.indices.refresh(index=index_name)

In [ ]:
client.count(index = index_name)

In [ ]:
qtxt = "finally gets a spare."

text_query = {
  "size": 5,
  "_source": ['title', 'description'],
  "query": {
    "multi_match": {
      "query": qtxt,
      "fields": ['description'],
    }
  }
}

response = client.search(
    body=text_query,
    index=index_name
)

print("\nSearch results:")
pprint(response)

Term-level Search

In [ ]:
term_query = {
    "size": 5,
    "_source": ["title", "description"],
    "query": {
        "term": {
            "title": "v_od9EdcDcByA"
        }
    }
}

response = client.search(
    body=term_query,
    index=index_name
)

print("\nSearch results:")
pprint(response)

Boolean Search

In [ ]:
bool_query = {
    "size": 5,
    "_source": ["title", "description"],
    "query": {
        "bool": {
            "must": [
                {"match": {"description": "skate"}},  # Must contain "skate"
                {"match": {"description": "tricks"}}  # Must contain "tricks"
            ],
        }
    }
}

response = client.search(
    body=bool_query,
    index=index_name
)

print("\nSearch results:")
pprint(response)

## 2.3 Embeddings Neighborhood

In [ ]:
client.indices.delete(index=index_name, ignore=[400, 404])

 ### Index mappings to support k-nn vectors

In [ ]:
index_body = {
    "settings": {
        "index": {
            "number_of_replicas": 0,
            "number_of_shards": 4,
            "refresh_interval": "-1",
            "knn": True
        }
    },
    "mappings": {
        "dynamic": "strict",
        "properties": {
            "title": { "type": "keyword" },
            "description": { "type": "text" },
            "sentence_embedding": {
                "type": "knn_vector",
                "dimension": 768,
                "method": {
                    "name": "hnsw",
                    "space_type": "innerproduct",
                    "engine": "faiss",
                    "parameters": {
                        "ef_construction": 256,
                        "m": 48
                    }
                }
            }
        }
    }
}


if client.indices.exists(index=index_name):
    print("Index already existed. You may force the new mappings.")
else:        
    response = client.indices.create(index_name, body=index_body)
    print('\nCreating index:')
    print(response)

In [ ]:
client.indices.refresh(index=index_name)

In [ ]:
client.count(index = index_name)

### Dual-Encoders

In [ ]:
tokenizer = AutoTokenizer.from_pretrained("sentence-transformers/msmarco-distilbert-base-v2")
model = AutoModel.from_pretrained("sentence-transformers/msmarco-distilbert-base-v2")

In [ ]:
#Mean Pooling - Take average of all tokens
def mean_pooling(model_output, attention_mask):
    token_embeddings = model_output.last_hidden_state #First element of model_output contains all token embeddings
    input_mask_expanded = attention_mask.unsqueeze(-1).expand(token_embeddings.size()).float()
    return torch.sum(token_embeddings * input_mask_expanded, 1) / torch.clamp(input_mask_expanded.sum(1), min=1e-9)


#Encode text
def encode(texts):
    # Tokenize sentences
    encoded_input = tokenizer(texts, padding=True, truncation=True, return_tensors='pt')

    # Compute token embeddings
    with torch.no_grad():
        model_output = model(**encoded_input, return_dict=True)

    # Perform pooling
    embeddings = mean_pooling(model_output, encoded_input['attention_mask'])

    # Normalize embeddings
    embeddings = F.normalize(embeddings, p=2, dim=1)
    
    return embeddings

Compute the video text embeddings

In [ ]:
all_embeddings = {}

for video_id, data in final_dataset_captions.items():
    # Join all sentences to one paragraph-like string
    full_description = " ".join(data['segments']['sentences'])
    
    # Encode the full description
    embedding = encode(full_description)
    
    all_embeddings[video_id] = {
        "title": video_id,
        "description": data['segments']['sentences'],
        "sentence_embedding": embedding[0].numpy()
    }
        
    resp = client.index(index=index_name, id=video_id, body=all_embeddings[video_id])
    print(resp['result'])

    stored = client.get(index=index_name, id=video_id)
    print("\nIndexed Document:")
    pprint(stored["_source"])
    print("-" * 50)

# Save the data to the pickle file
with open('all_embeddings.pkl', 'wb') as f:
    pickle.dump(all_embeddings, f)

### Load the data from the pickle file

In [ ]:

with open('all_embeddings.pkl', 'rb') as f:
    all_embeddings = pickle.load(f)
print(all_embeddings)


### Search query for embedding index

In [ ]:
client.indices.refresh(index=index_name)

In [ ]:
query = "finally gets a spare."
query_emb = encode(query)

query_denc = {
  'size': 5,
  '_source': ['title', 'description'],
   "query": {
        "knn": {
          "sentence_embedding": {
            "vector": query_emb[0].numpy(),
            "k": 2
          }
        }
      }
}

response = client.search(
    body = query_denc,
    index = index_name
)

print('\nSearch results:')
pprint(response)

Discuss how the embeddings space organize data and allow for specific search:

### 1. Data Organization in Embedding Space
In **semantic search**, each document is transformed into a vector and positioned within a **high-dimensional space**. Documents with similar meanings are placed **close together**, regardless of the actual words used.

In **traditional search**, documents are stored based on **text terms and frequencies**. The index organizes data around **keyword occurrence**, not meaning.

### 2. Proximity-Based Search
In the **embedding space**, search is performed by finding documents that are **nearest neighbors** to the query vector. This means results are selected based on **semantic similarity**, not shared vocabulary.

In **traditional search**, relevance is determined by **keyword overlap and frequency**. The system doesn’t consider whether two different words or phrases mean the same thing.

### 3. Flexibility in Querying
**Semantic search** enables queries to return documents based on the **meaning alignment**, even if no keywords match directly.

**Traditional search** would require the exact terms to appear in the document to be considered relevant, limiting its ability to understand **context or nuance**.

### 4. Dimensional Context
The **embedding space** captures **contextual relationships** across hundreds of dimensions. This allows it to distinguish between similar words used in **different contexts** (e.g., `"apple"` the fruit vs. `"Apple"` the company).

**Traditional indexing** does not capture this level of nuance—it treats words more **literally and statically**.


## 2.4 Constrained Embedding Searches

In [ ]:
nlp = spacy.load("en_core_web_sm")
query = "show me moments of a man surfing."
query_embedding = encode(query)[0].numpy().tolist()

def get_nouns_verbs(text):
    doc = nlp(text)
    return " ".join([token.text for token in doc if token.pos_ in {"NOUN", "VERB"}])

filter_terms = get_nouns_verbs(query)
print(f"Filter terms: {filter_terms}")

search_body = {
    'size': 100,
    "query": {
        "bool": {
            "must": [{
                "knn": {
                    "sentence_embedding": {
                        "vector": query_embedding,
                        "k": 1
                    }
                }
            }],
            "filter": [{
                "match": {
                    "description": {
                        "query": filter_terms,
                        "operator": "or"
                    }
                }
            }]
        }
    },
    "_source": ["title", "description"]
}

response = client.search(index=index_name, body=search_body)
print(f"Total documents found: {response['hits']['total']['value']}")

for hit in response["hits"]["hits"]:
    print(f"Title: {hit['_source']['title']}")
    print("Relevant sentences:")
    for sentence in hit['_source']['description']:
        if "surf" in sentence.lower():
            print(f"  → {sentence}")
        else:
            print(f"  - {sentence}")


In [ ]:
query = "bokling"  # Intentional typo for "bowling"

search_body = {
    "query": {
        "match": {
            "description": {
                "query": query,
                "fuzziness": "AUTO",  # Automatically corrects "bokling" to "bowling"
                "operator": "or"
            }
        }
    }
}

response = client.search(index=index_name, body=search_body)
print(f"Total documents found: {response['hits']['total']['value']}")

for hit in response["hits"]["hits"]:
    print(f"Title: {hit['_source']['title']}")
    print("Full description:")
    for sentence in hit['_source']['description']:
        print(f"  - {sentence}")


## 2.5 Contextual embeddings and Self-attention

### Contextual embeddings

In [ ]:
sentence = "He raises his hands feeling victorious."

In [ ]:
model_path = 'nboost/pt-bert-base-uncased-msmarco'

tokenizer = AutoTokenizer.from_pretrained(model_path)
config = AutoConfig.from_pretrained(model_path,  output_hidden_states=True, output_attentions=True)  
model = AutoModelForSequenceClassification.from_pretrained(model_path, config=config)

inputs = tokenizer(sentence, return_tensors='pt', add_special_tokens=True, max_length=512, padding=True, truncation=True)
input_ids = inputs["input_ids"][0]
input_tokens_list = tokenizer.convert_ids_to_tokens(input_ids)
with torch.no_grad():
    outputs = model(**inputs)

Hidden layer embedding

In [ ]:
# The format is as follow:
# outputs['hidden_states'][layer_m][0][token_n]
layer_m = 12
token_n = 1
# Get all the embeddings of one layer:
output_embeddings = outputs['hidden_states'][layer_m][0]
output_embeddings.shape

hidden_states = outputs.hidden_states

In [ ]:
def display_scatterplot(data, words):

    if data.shape[1] == 2:
        twodim = data
    else:
        pca = PCA()
        pca.fit(output_embeddings.detach().numpy())
        twodim = pca.transform(data)[:,:2]
    
    plt.style.use('default') # https://matplotlib.org/3.5.1/gallery/style_sheets/style_sheets_reference.html
    plt.figure(figsize=(6,6))
    plt.scatter(twodim[:,0], twodim[:,1], edgecolors='k', c='r')
    for word, (x,y) in zip(words, twodim):
        plt.text(x+0.05, y+0.05, word)

    return

display_scatterplot(output_embeddings.detach().numpy(), input_tokens_list)

In [ ]:
layer = 1

rows = 3
cols = 4
fig, ax_full = plt.subplots(rows, cols)
fig.set_figheight(rows*4)
fig.set_figwidth(cols*4+3)
plt.rcParams.update({'font.size': 6})


layer = 0
for r in range(rows):
    for c in range(cols):
       
        ax = ax_full[r,c]
        
        plt.rcParams.update({'font.size': 10})
        current_hidden_state = hidden_states[layer][0].detach().numpy()
        
        if current_hidden_state.shape[1] == 2:
            twodim = current_hidden_state
        else:
            twodim = PCA().fit_transform(current_hidden_state)[:,:2]

        plt.style.use('default') # https://matplotlib.org/3.5.1/gallery/style_sheets/style_sheets_reference.html
        im = ax.scatter(twodim[:,0], twodim[:,1], edgecolors='k', c='r')
        for word, (x,y) in zip(input_tokens_list, twodim):
            ax.text(x+0.05, y+0.05, word[1:])
        
        # Show all ticks and label them with the respective list entries
        ax.set_title("Layer " + str(layer))
            
        # Loop over data dimensions and create text annotations.
        layer = layer + 1

fig.suptitle("Visualization of all output embeddings from all layers")
plt.show()

#### Key Observations

1. **Layer 0: Initial Embeddings (Non-contextualized)**
   - Tokens are close together and not semantically distinct.
   - This layer contains basic wordpiece and position embeddings before any attention is applied.

2. **Middle Layers (Layers 3–7): Emerging Context**
   - Tokens begin to spread out.
   - Contextual differences become visible — e.g., `"victorious"` starts to drift away from `"hands"` or `"his"`.
   - The model begins to shape representations based on surrounding words.

3. **Deeper Layers (Layers 8–11): Fully Contextualized Embeddings**
   - Tokens are well-separated and semantically meaningful.
   - Embeddings reflect deep contextual understanding (e.g., `"hands"` vs. `"victorious"` are clearly distinct).
   - Special tokens like `[CLS]` and `[SEP]` are positioned separately, indicating their functional role in the architecture.

4. **Token Stability vs. Drift**
   - Some tokens (e.g., `"he"`, `"hands"`) move significantly across layers.
   - Special tokens (e.g., `[SEP]`) remain relatively stable, as their role is consistent.

---

#### Conclusion

- The embeddings start out **non-contextualized** and become **increasingly informed by sentence context** as we move up the layers.
- This layered transformation allows BERT to encode rich semantic relationships, which are crucial for tasks like classification, ranking, or semantic search.
- Visualizing these embeddings reveals how each layer contributes to building up meaning, culminating in task-specific representations in the final layers.

### Positional embeddings

In [ ]:
sentence = "hello hello hello hello hello hello hello hello hello hello hello hello hello hello hello hello hello hello hello hello"

In [ ]:
model_path = 'nboost/pt-bert-base-uncased-msmarco'

tokenizer = AutoTokenizer.from_pretrained(model_path)
config = AutoConfig.from_pretrained(model_path,  output_hidden_states=True, output_attentions=True)  
model = AutoModelForSequenceClassification.from_pretrained(model_path, config=config)

inputs = tokenizer(sentence, return_tensors='pt', add_special_tokens=True, max_length=512, padding=True, truncation=True)
input_ids = inputs["input_ids"][0]
input_tokens_list = tokenizer.convert_ids_to_tokens(input_ids)
with torch.no_grad():
    outputs = model(**inputs)

In [ ]:
# The format is as follow:
# outputs['hidden_states'][layer_m][0][token_n]
layer_m = 12
token_n = 1
# Get all the embeddings of one layer:
output_embeddings = outputs['hidden_states'][layer_m][0]
output_embeddings.shape

hidden_states = outputs.hidden_states

In [ ]:
def display_scatterplot(data, words):

    if data.shape[1] == 2:
        twodim = data
    else:
        pca = PCA()
        pca.fit(output_embeddings.detach().numpy())
        twodim = pca.transform(data)[:,:2]
    
    plt.style.use('default') # https://matplotlib.org/3.5.1/gallery/style_sheets/style_sheets_reference.html
    plt.figure(figsize=(6,6))
    plt.scatter(twodim[:,0], twodim[:,1], edgecolors='k', c='r')
    for word, (x,y) in zip(words, twodim):
        plt.text(x+0.05, y+0.05, word)

    return

display_scatterplot(output_embeddings.detach().numpy(), input_tokens_list)

In [ ]:
layer = 1

rows = 3
cols = 4
fig, ax_full = plt.subplots(rows, cols)
fig.set_figheight(rows*4)
fig.set_figwidth(cols*4+3)
plt.rcParams.update({'font.size': 6})


layer = 0
for r in range(rows):
    for c in range(cols):
       
        ax = ax_full[r,c]
        
        plt.rcParams.update({'font.size': 10})
        current_hidden_state = hidden_states[layer][0].detach().numpy()
        
        if current_hidden_state.shape[1] == 2:
            twodim = current_hidden_state
        else:
            twodim = PCA().fit_transform(current_hidden_state)[:,:2]

        plt.style.use('default') # https://matplotlib.org/3.5.1/gallery/style_sheets/style_sheets_reference.html
        im = ax.scatter(twodim[:,0], twodim[:,1], edgecolors='k', c='r')
        for word, (x,y) in zip(input_tokens_list, twodim):
            ax.text(x+0.05, y+0.05, word[1:])
        
        # Show all ticks and label them with the respective list entries
        ax.set_title("Layer " + str(layer))
            
        # Loop over data dimensions and create text annotations.
        layer = layer + 1

fig.suptitle("Visualization of all output embeddings from all layers")
plt.show()

#### Key Observations

1. **Token Uniqueness via Position**
   - Even though all tokens are `"hello"`, their embeddings differ.
   - This difference is due to **positional encodings** added before the first layer.

2. **Early Layer Behavior**
   - In Layer 0, the tokens already diverge — showing that **positional encoding is added at the embedding stage**.

3. **Later Layers Reveal Positional Structure**
   - From Layer 8 onward, tokens are organized in **structured arcs**, reflecting their **relative positions** in the sentence.

4. **Special Tokens Are Isolated**
   - `[CLS]` and `[SEP]` tokens are consistently separated, showing they are **functionally unique**.

---

#### Conclusion

Transformers successfully encode **position** via embeddings, even for identical tokens. Over layers, the network maintains and enhances this information, enabling it to understand **word order and relative position** — a crucial property for handling natural language sequences.

### Self-attention

Cross-encoder

In [ ]:
def call_html():
  import IPython
  display(IPython.core.display.HTML('''
        <script src="/static/components/requirejs/require.js"></script>
        <script>
          requirejs.config({
            paths: {
              base: '/static/base',
              "d3": "https://cdnjs.cloudflare.com/ajax/libs/d3/5.7.0/d3.min",
              jquery: '//ajax.googleapis.com/ajax/libs/jquery/2.0.0/jquery.min',
            },
          });
        </script>
        '''))
    

In [ ]:
sentence = "He raises his hands feeling victorious."
question = "What did he raise?"

In [ ]:
model_path = 'nboost/pt-bert-base-uncased-msmarco'

tokenizer = AutoTokenizer.from_pretrained(model_path)
config = AutoConfig.from_pretrained(model_path,  output_hidden_states=True, output_attentions=True)  
model = AutoModel.from_pretrained(model_path, config=config)

inputs = tokenizer(sentence, return_tensors='pt', add_special_tokens=True, max_length=512, padding=True, truncation=True)
input_ids = inputs["input_ids"][0]
tokens = tokenizer.convert_ids_to_tokens(input_ids)

with torch.no_grad():
    outputs = model(**inputs)

attentions = outputs.attentions

Tokens specific visualization of self-attention

In [ ]:
call_html()
head_view(attentions, tokens)

Visualization of all the attention heads in one layer

In [ ]:
layer = 1

rows = 3
cols = 4
fig, ax_full = plt.subplots(rows, cols)
fig.set_figheight(rows*6)
fig.set_figwidth(cols*6+4)
plt.rcParams.update({'font.size': 10})

j = 0
for r in range(rows):
    for c in range(cols):
       
        ax = ax_full[r,c]
        
        sattention = attentions[layer][0][j].numpy()
        sattention = np.flip(sattention, 0)
        
        plt.rcParams.update({'font.size': 10})

        im = ax.pcolormesh(sattention, cmap='gnuplot')

        # Show all ticks and label them with the respective list entries
        ax.set_title("Head " + str(j))
        ax.set_yticks(np.arange(len(tokens)))
        if c == 0:
            ax.set_yticklabels(reversed(tokens))
            ax.set_ylabel("Queries")
        else:
            ax.set_yticks([])

        ax.set_xticks(np.arange(len(tokens)))
        if r == rows-1:
            ax.set_xticklabels(tokens)
            ax.set_xlabel("Keys")
            
            # Rotate the tick labels and set their alignment.
            plt.setp(ax.get_xticklabels(), rotation=45, ha="right",
                     rotation_mode="anchor")
        else:
            ax.set_xticks([])

            
        # Loop over data dimensions and create text annotations.
        j = j + 1

fig.suptitle("Layer" + str(layer) + " Multi-head Self-attentions")
cbar = fig.colorbar(im, ax=ax_full, location='right', shrink=0.5)
cbar.ax.set_ylabel("Selt-attention", rotation=-90, va="bottom")
plt.show()

Visualzation of Self-Attention in all layers and heads

In [ ]:
model_view(attentions, tokens)

#### Dual-encoder

In [ ]:
inputs_a = tokenizer(question, return_tensors="pt")
inputs_b = tokenizer(sentence, return_tensors="pt")

with torch.no_grad():
    output_a = model(**inputs_a).last_hidden_state.squeeze(0)
    output_b = model(**inputs_b).last_hidden_state.squeeze(0) 

output_a_norm = torch.nn.functional.normalize(output_a, p=2, dim=1)
output_b_norm = torch.nn.functional.normalize(output_b, p=2, dim=1)

similarity_matrix = torch.matmul(output_a_norm, output_b_norm.T)  # (seq_len_a, seq_len_b)

tokens_a = tokenizer.convert_ids_to_tokens(inputs_a["input_ids"][0])
tokens_b = tokenizer.convert_ids_to_tokens(inputs_b["input_ids"][0])

plt.figure(figsize=(10, 6))
sns.heatmap(similarity_matrix.numpy(), xticklabels=tokens_b, yticklabels=tokens_a, cmap="viridis", annot=True, fmt=".2f")
plt.xlabel("Sentence Tokens")
plt.ylabel("Question Tokens")
plt.title("Similarity between Question and Sentence Tokens")
plt.show()

#### Cross-Encoder

In the cross-encoder, **all tokens in the input sequence are encoded together**, allowing **token-level interactions** via the self-attention mechanism.

**Key Observations:**
- Attention heatmaps clearly show **which tokens attend to which others**.
- Special tokens like `[CLS]` often attend to semantically rich tokens (e.g., "victorious", "hands").
- Later layers show **sharper, more focused attention** — indicating higher-level understanding.
- Attention heads behave differently — some focus on syntax (e.g., function words), others on semantics (e.g., verbs or sentiment words).

**Conclusion**: Cross-encoders capture **fine-grained relationships** between words, making them highly effective for tasks requiring deep understanding, like natural language inference or question answering.

---

#### Dual Encoder

In the dual encoder, **each sentence is encoded independently**, without any interaction at the token level.

**Key Observations:**
- No attention is computed between tokens across sentences — each input is processed in isolation.
- No attention heatmaps can be generated between sentence pairs.
- Only final sentence embeddings (pooled outputs) are compared — typically using **cosine similarity**.
- The model prioritizes **speed and scalability** over fine-grained interaction.

**Conclusion**: Dual encoders are ideal for **semantic similarity and retrieval tasks**, where you need fast comparison of many sentences — but they lack interpretability and detailed attention dynamics.

---

#### Multi-Head Self-Attention using Cross-Encoder

**Key Observations:**

- **Self-Attention Dominance**: Most heads (e.g., Head 0, Head 4) show strong diagonal patterns, indicating tokens primarily attend to themselves.
  
- **Special Token Behavior**:
  - `[CLS]` tokens (Heads 0, 3, 6) attend broadly, aggregating global sentence information.
  - `[SET]` tokens (Heads 2, 5) focus heavily on "victorious", suggesting a role in capturing sentence-final semantics.

- **Action-Emotion Links**:
  - Heads 1 and 6 connect "raises" to "hands" (action-object) and "feeling" to "victorious" (emotion-result).

- **Underutilized Heads**: Heads 9–11 exhibit weak/no meaningful attention, potentially redundant for this input.

**Conclusion**: The model distributes attention across heads to balance local token relationships (e.g., verb-noun pairs) and global sentence context (via `[CLS]`/`[SET]`). While some heads specialize in semantic or structural roles, others contribute minimally, suggesting room for optimization.

---

#### Token-Level Similarity in Dual Encoders

Although dual encoders do not provide cross-token attention, we can still analyze **semantic alignment between tokens** using cosine similarity between token embeddings of each sentence.

**Key Observations:**
- Pronouns such as `"he"` in the question tend to align closely with `"he"` in the sentence, indicating proper referential understanding.
- Verbs like `"raise"` show strong similarity with their conjugated forms like `"raises"`, reflecting the model's ability to capture morphological relationships.
- `[CLS]` tokens are highly similar, as they both represent global sentence-level meaning.
- Punctuation and structural tokens like `[SEP]` and `"."` often have low similarity with semantic tokens, though they may align with each other due to similar positional roles.

**Conclusion**: While dual encoders do not compute attention between sequences, analyzing token embedding similarity reveals that the model still encodes meaningful semantic relationships. This can provide interpretability at a coarse level, especially useful for understanding what drives the final sentence embedding similarity.

---

#### Critical Comparison

| Feature                        | Cross-Encoder                  | Dual Encoder                        |
|-------------------------------|--------------------------------|-------------------------------------|
| Token-level attention         | Yes                            | No                                  |
| Inter-sentence interaction    | Yes                            | No                                  |
| Interpretability (heatmaps)   | High                           | Limited (via cosine similarity)     |
| Computational cost            | Higher                         | Lower                               |
| Use case                      | QA, NLI, classification        | Search, retrieval, ranking          |
| Representation granularity    | Fine-grained (token-level)     | Coarse (sentence-level only)        |

---

#### Final Thoughts

- **Cross-encoders** provide rich interpretability through attention maps and capture deep contextual relationships — but are computationally expensive.
- **Dual encoders** are lightweight and fast, making them practical for large-scale applications — but offer no visibility into token-level interactions.
- The choice between them depends on the **task requirements**: precision vs. speed, interpretability vs. scalability.

### Interpretability

In [ ]:
layer = 11  # analyze the final layer
attentions = outputs.attentions  # shape: (num_layers, batch, num_heads, seq_len, seq_len)

# Get shape info
num_layers = len(attentions)
seq_len = attentions[0].shape[-1]

# Store total attention received for each token across all heads, per layer
attention_received = torch.zeros((num_layers, seq_len))

# Loop through each layer
for l in range(num_layers):
    attn = attentions[l][0]  # shape: (num_heads, seq_len, seq_len)
    attn_sum = attn.sum(dim=0)  # sum over query dimension → shape: (seq_len, seq_len)
    attention_received[l] = attn_sum.sum(dim=0)  # sum over key dimension


In [ ]:
import matplotlib.pyplot as plt

tokens = tokenizer.convert_ids_to_tokens(inputs['input_ids'][0])

plt.figure(figsize=(10, 6))
for token_idx in range(seq_len):
    plt.plot(range(num_layers), attention_received[:, token_idx], label=tokens[token_idx])

plt.title("Total Attention Received by Each Token Across Layers")
plt.xlabel("Layer")
plt.ylabel("Total Attention Received")
plt.legend(bbox_to_anchor=(1.05, 1), loc='upper left')
plt.grid(True)
plt.tight_layout()
plt.show()


### Interpretability: Attention Received by Tokens Across Layers

The plot shows how much **total attention each token receives** across the 12 transformer layers for the sentence:

> **"He raises his hands feeling victorious."**

Each line represents a token, and its value at each layer indicates how much attention it received from all other tokens combined (summed across all attention heads).

---

#### Key Observations

1. **[CLS] Token Behavior**
   - Receives a significant amount of attention in **early (Layers 0–2)** and **late (Layers 10–11)** stages.
   - This aligns with its role as the **summary token**, aggregating sentence-level meaning for classification tasks.

2. **[SEP] Token Behavior**
   - Dominates the middle layers (especially **Layers 4–9**) in terms of attention received.
   - This might reflect its role in separating segments or as a structural marker the model relies on to anchor information.

3. **"he" Token Stands Out**
   - The token `"he"` receives unusually high attention in **Layers 5–9**, surpassing even structural tokens.
   - This suggests the model identifies `"he"` as a central semantic component, possibly due to it being the subject of the sentence.

4. **Content Words (e.g., "hands", "victorious")**
   - Receive **moderate and stable attention** throughout the layers.
   - Tokens like `"victorious"` and `"raises"` show some variance, indicating they contribute to the evolving meaning but aren’t the primary focus of attention.

5. **Function Words (e.g., "his", "feeling", ".")**
   - These tokens receive consistently **low attention**, showing they are less critical in shaping the final output.

---

#### Conclusion

- Tokens that are **structurally important** (`[CLS]`, `[SEP]`) and **semantically central** (`"he"`) receive the **highest total attention**.
- The model learns to focus its attention **dynamically across layers**, initially on structure, then content, and finally back to the `[CLS]` token for producing the final output.
- This behavior confirms that attention received is a strong indicator of **token importance**, which helps explain **how the model constructs meaning** layer by layer.

